In [21]:
import spotipy
import pandas as pd

from sqlalchemy import create_engine
from spotipy.oauth2 import SpotifyOAuth

from config import CLIENT_ID, CLIENT_SECRET, REDIRECT_URI, SCOPE, DATABASE_URL

from utils import log_exception

In [22]:
# ETL functions related to user's top artists

def extract_top_artists():

    try:
        # Authentication with Spotify API
        sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                                       client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope=SCOPE))
        raw_data = sp.current_user_top_artists()
        return raw_data
    except Exception as e:
        log_exception("Error while extracting user's top artists data:", e)
        return None


def transform_top_artists(raw_data):

    try:
        user_top_artists = raw_data["items"]

        df = pd.DataFrame(data=user_top_artists)
        artists_df = df[["id", "name", "popularity"]]

        return artists_df
    except Exception as e:
        log_exception("Error while transforming user's top artists data:", e)
        return None


def load_top_artists(transformed_data):

    success = False
    try:
        engine = create_engine(url=DATABASE_URL)
        transformed_data.to_sql(name="user_top_artists",
                                con=engine, if_exists="replace", index=False)

        success = True
    except Exception as e:
        log_exception("Error while loading user's top artists data:", e)
        return None
    finally:
        return success


def run_top_artists_etl():

    success = False
    try:
        raw_data = extract_top_artists()

        transformed_data = transform_top_artists(raw_data)

        success = load_top_artists(transformed_data)

    except Exception as e:
        log_exception(
            "Error while attempting to run user's top artists ETL:", e)
    finally:
        return success

In [23]:
# ETL functions related to user's top songs

def extract_top_songs():
    try:
        sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                                       client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope=SCOPE))

        raw_data = sp.current_user_top_tracks()

        return raw_data
    except Exception as e:
        log_exception(
            "Error while attempting to extract user's top songs: ", e)
        return None


def transform_top_songs(raw_data):
    try:
        user_top_songs = raw_data["items"]

        df = pd.DataFrame(data=user_top_songs)

        songs_df = df[["id", "name", "popularity"]]

        track_ids = songs_df["id"].tolist()

        sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                                       client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope=SCOPE))

        audio_features = sp.audio_features(tracks=track_ids)

        df = pd.DataFrame(data=audio_features)

        analysis_df = df[["duration_ms", "key", "tempo"]]

        combined_df = pd.concat([songs_df, analysis_df], axis=1)

        return combined_df
    except Exception as e:
        log_exception(
            "Error while attempting to transform user's top songs: ", e)
        return None


def load_top_songs(transformed_data):
    success = False
    try:
        engine = create_engine(url=DATABASE_URL)

        transformed_data.to_sql(name="user_top_songs",
                                con=engine, if_exists="replace", index=False)
        success = True
    except Exception as e:
        log_exception("Error while attempting to load user's top songs: ", e)
    finally:
        return success


def run_top_songs_etl():
    success = False
    try:
        raw_data = extract_top_songs()

        transformed_data = transform_top_songs(raw_data)

        success = load_top_songs(transformed_data)
    except Exception as e:
        log_exception(
            "Error while attempting to run user's top songs ETL: ", e)
    finally:
        return success

In [24]:
run_top_artists_etl()
run_top_songs_etl()

True